In [1]:
import polars as pl
from polars import DataFrame

from src.dog_data_analysis import normalize_breed_names, licenses_by_breed, extract_unique_breeds

bronze_data_path:str = './data/bronze/2017.csv'
silver_data_path:str = './data/silver/2017.csv'

# Load lazy CSV data
bronze_dog_data:pl.LazyFrame = pl.scan_csv(bronze_data_path, separator=',', has_header=True)
silver_dog_data:pl.LazyFrame = pl.scan_csv(silver_data_path, separator=',', has_header=True)
print(bronze_dog_data.head(2))

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SLICE[offset: 0, len: 2]
  Csv SCAN [./data/bronze/2017.csv]
  PROJECT */7 COLUMNS


In [2]:
# Part 1: Normalize Breeds 
normalized_data = normalize_breed_names(bronze_dog_data)
normalized_data.write_csv(silver_data_path, include_header=True)
print("Normalized Data:", normalized_data)

Normalized Data: shape: (21_727, 8)
┌────────────┬────────────┬────────────┬────────────┬──────────┬─────────┬────────────┬────────────┐
│ LicenseTyp ┆ Breed      ┆ Color      ┆ DogName    ┆ OwnerZip ┆ ExpYear ┆ ValidDate  ┆ normalized │
│ e          ┆ ---        ┆ ---        ┆ ---        ┆ ---      ┆ ---     ┆ ---        ┆ _breed     │
│ ---        ┆ str        ┆ str        ┆ str        ┆ i64      ┆ i64     ┆ str        ┆ ---        │
│ str        ┆            ┆            ┆            ┆          ┆         ┆            ┆ str        │
╞════════════╪════════════╪════════════╪════════════╪══════════╪═════════╪════════════╪════════════╡
│ Dog        ┆ BICHON     ┆ WHITE      ┆ CHLOE      ┆ 15090    ┆ 2017    ┆ 12/15/2016 ┆ bichon     │
│ Individual ┆ FRISE      ┆            ┆            ┆          ┆         ┆ 9:58       ┆ frise      │
│ Spayed     ┆            ┆            ┆            ┆          ┆         ┆            ┆            │
│ Female     ┆            ┆            ┆            ┆  

In [6]:
# Find Unique Breeds
unique_breeds = extract_unique_breeds(normalized_data)
print("Unique Breeds (SQL):", unique_breeds)
unique_breeds.write_csv("./data/silver/unique_breeds.csv", include_header=True)

Unique Breeds (SQL): shape: (265, 1)
┌─────────────────────┐
│ normalized_breed    │
│ ---                 │
│ str                 │
╞═════════════════════╡
│ bichon frise        │
│ chihuahua           │
│ lab mix             │
│ dachshund           │
│ terrier             │
│ …                   │
│ boerboel            │
│ pumi                │
│ am black&tan coonho │
│ carolina dog        │
│ jindo               │
└─────────────────────┘


In [4]:
# Part 2: Counting Licenses by License Type
license_counts = licenses_by_breed(silver_dog_data)
print("License Counts by License Type and Breed (SQL):\n", license_counts)

ColumnNotFoundError: Breed

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'select' <---
AGGREGATE
  [len().alias("license_count")] BY [col("LicenseType"), col("normalized_breed")]
  FROM
  Csv SCAN [./data/silver/2017.csv]
  PROJECT */8 COLUMNS